In [2]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from scipy.signal import decimate

In [3]:
class DataProcessor:
    def __init__(self, input_path, file_names):
        self.input_path = input_path
        self.file_names = file_names
        
    def read_files(self):
        self.data = {}
        print("Reading files...")
        for file in self.file_names:
            with open(self.input_path + file + '.txt', 'r') as f:
                self.data[file] = pd.read_csv(f, header=None, sep='\t')
        return self.data
    
    def print_shape(self):
        print("Files read:")
        for file in self.data:
            print(f"{file}: {self.data[file].shape}")
            
    def create_target_df(self):
        target_columns = ['Cooler_Condition', 'Valve_Condition', 
                        'Internal_Pump_Leakage', 'Hydraulic_Accumulator', 
                        'Stable_Flag']
        self.data['target'].columns = target_columns
        self.valve_condition = self.data['target']['Valve_Condition']
        #del self.data['target']
        return self.valve_condition

def process_data():
    input_path = "input_data/"
    file_names = [
        "ce", "cp", "eps1", "se", "vs1", 
        "fs1", "fs2", 
        "ps1", "ps2", "ps3", "ps4", "ps5", "ps6",
        "ts1", "ts2", "ts3", "ts4", "target"
    ]
    
    processor = DataProcessor(input_path, file_names)
    data = processor.read_files()
    processor.print_shape()
    df_target = processor.create_target_df()
    df_target = processor.valve_condition
    return data, df_target

data, df_target = process_data()

Reading files...
Files read:
ce: (2205, 60)
cp: (2205, 60)
eps1: (2205, 6000)
se: (2205, 60)
vs1: (2205, 60)
fs1: (2205, 600)
fs2: (2205, 600)
ps1: (2205, 6000)
ps2: (2205, 6000)
ps3: (2205, 6000)
ps4: (2205, 6000)
ps5: (2205, 6000)
ps6: (2205, 6000)
ts1: (2205, 60)
ts2: (2205, 60)
ts3: (2205, 60)
ts4: (2205, 60)
target: (2205, 5)


In [4]:
input_df = pd.concat([data['ce'], data['cp'], data['eps1'], data['se'], data['vs1'], 
                      data['fs1'], data['fs2'], data['ps1'], data['ps2'], data['ps3'], data['ps4'], 
                      data['ps5'], data['ps6'], data['ts1'], data['ts2'], data['ts3'], data['ts4']], axis=1)

In [93]:
df_list = ['ce', 'cp', 'eps1', 'se', 'vs1', 'fs1', 'fs2', 'ps1', 'ps2', 'ps3', 'ps4', 'ps5', 'ps6']
input_df = pd.concat([data[i] for i in df_list], axis = 1)

In [5]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(df_target)

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.optimizers import Adam

def create_model(input_dim):
    model = Sequential([
        # Erste Convolutional Layer (1D)
        Conv1D(64, 3, activation='relu', input_shape=(input_dim, 1)),  # input_dim=68000, 1 Kanal für flache Daten
        MaxPooling1D(pool_size=2),  # Max Pooling zur Reduktion der Dimensionalität
    
        #MaxPooling1D(pool_size=2),
        
        # Flatten und Fully Connected Layers
        Flatten(),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(16, activation='relu'),
        Dense(4, activation='softmax')  # 4 Klassen für 'Valve Condition'
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model


In [7]:
states = [27, 6728, 49122]
target = y_encoded

In [9]:
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

input_data = input_df  # Ensure this is defined  # Define your list of random states (or use the actual values you need)

accs = []  # List to store accuracy scores if needed

for RANDOM_STATE in states:
    # Datenaufteilung und Standardisierung
    X_train, X_test, y_train, y_test = train_test_split(
        input_data, target, test_size=0.2, random_state=RANDOM_STATE, stratify=target
    )
    
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    # Modell erstellen und trainieren
    model = create_model(input_dim=X_train.shape[1])
    
    history = model.fit(
        X_train, y_train,
        validation_split=0.2,
        epochs=10,
        batch_size=32,
        verbose=0
    )

    # Vorhersagen und Genauigkeit berechnen
    y_proba = model.predict(X_test)
    y_pred = y_proba.argmax(axis=1)
    
    # Accuracies if you want to store them for later use
    accs.append(accuracy_score(y_test, y_pred))
    
    # Ergebnisse ausgeben
    print(f"Random State: {RANDOM_STATE}")
    print(classification_report(y_test, y_pred))

# Durchschnittliche und Standardabweichung der Genauigkeiten
accs_mean = round(np.mean(accs), 4)  # Mean accuracy
accs_std = round(np.std(accs), 4)  # Standard deviation

print(f"Mean Accuracy: {accs_mean}")
print(f"Std Accuracy: {accs_std}")


c:\Users\Daniel\anaconda3\envs\tf_env\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


KeyboardInterrupt: 